In [1]:
import pandas as pd
from sklearn.neighbors import KernelDensity
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pdf = pd.read_csv('pulls_data_clean.csv')
mdf = gpd.read_file('milemarkers.gdb')
vdf = pd.read_csv('TxDOT_AADT_Annuals.csv')


/var/folders/fz/qwk5rxqs39gcy8h_0lhy47c80000gn/T/ipykernel_40576/2240971900.py:9: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  pdf = pd.read_csv('pulls_data_clean.csv')


In [2]:
road_type_to_abbreviation = {
    'INTERSTATE': 'IH',
    'US HIGHWAY': 'US',
    'STATE HIGHWAY': 'SH',
    'FARM TO MARKET': 'FM',
    'FARM TO MARKET ROAD SPUR': 'FS',
    'STATE HIGHWAY SPUR': 'SS',
    'TOLLWAY': None,  
    'OFF INTERSTATE BUS. RTE.': None, 
    'RECREATION ROAD': 'RE',
    'STATE HIGHWAY LOOP': 'SL',
    'OFF US HWY. BUS. RTE.': 'BU', 
    'RANCH TO MARKET': 'RM',
    'US ALTERNATE': 'UA',
    'LOOP': 'SL',  
    'SPUR': 'SS',  
    'STATE HIGHWAY ALTERNATE': None,  
    'OFF ST. HWY. BUS. RTE.': 'BS', 
    'PARK ROAD': 'PR',
    'TEXAS STATE HIGHWAY': 'SH', 
    'US HIGHWAY SPUR': 'UP',
    'PORT OF ENTRY': None  
}

pdf['abr_type'] = pdf['Route_Type'].map(road_type_to_abbreviation)
pdf = pdf.dropna(subset='abr_type')
pdf['road'] = (pdf['abr_type'] + pdf['Route']).str.strip()

def pad_zeroes(input_str):
    num_zeroes = 4 - len(input_str)
    return "0" * num_zeroes + input_str
mdf['road'] = mdf['RTE_PRFX'] + mdf['RTE_NBR'].astype(str).apply(pad_zeroes)

vcounts = vdf.groupby('ON_ROAD').count()
vroads = vcounts[vcounts['X']>5].index.to_list()
vroads = [s[:6] for s in vroads]
roads = list(set(pdf['road'].unique()) & set(vroads) & set(mdf['road'].unique()))


In [3]:
from scipy.interpolate import InterpolatedUnivariateSpline

def interpolate_coordinates(mile1, mile2, coord1, coord2, mile):
    fraction = (mile - mile1) / (mile2 - mile1)
    x = coord1.x + fraction * (coord2.x - coord1.x)
    y = coord1.y + fraction * (coord2.y - coord1.y)
    return Point(x, y)

def closest_mile_marker(x,y, df):
    coord = Point(x,y)
    distances = df['Coordinates'].apply(lambda point: point.distance(coord))
    idx = distances.idxmin()
    return df.loc[idx, 'Mile']

def first_non_number(s):
    for char in s:
        if not char.isnumeric():
            return char
    return None

def check_ratio(series):
    ratio_forward = series.shift(-1) / series
    ratio_backward = series.shift() / series

    mask = ((ratio_forward >= 10) | (ratio_forward <= 0.1) |
            (ratio_backward >= 10) | (ratio_backward <= 0.1))
    return mask

def remove_outliers(df, column_name, x=2):
    mean_val = df[column_name].mean()
    std_dev = df[column_name].std()
    
    lower_bound = mean_val - x * std_dev
    upper_bound = mean_val + x * std_dev
    
    filtered_df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]
    
    return filtered_df

def remove_outliers_by_window(df, n=15, x=2):
    mean = df['AADT_2022'].rolling(window=n, center=True).mean()
    std = df['AADT_2022'].rolling(window=n, center=True).std()
    mask = (df['AADT_2022'] - mean).abs() <= x * std
    return df[mask]

dfs = []
for i in range(len(roads)):
    road = roads[i]
    print(road)

    tpdf = pdf[pdf['road']==road]
    tpdf = tpdf.sort_values('Milepost')

    tmdf = mdf[mdf['road']==road]
    tmdf = tmdf.sort_values('MRKR_NBR')
    if 'KG' in list(tmdf['RDBD_TYPE'].unique()):
        tmdf = tmdf[tmdf['RDBD_TYPE']=='KG']

    tvdf = vdf[vdf['ON_ROAD']==road]
    if len(tvdf.index) < 2:
        print('bad df')
        continue

    X = tpdf['Milepost'].values.reshape(-1, 1)
    Xlist = tmdf['MRKR_NBR'].values
    kde = KernelDensity(kernel='gaussian', bandwidth=1)
    kde.fit(X)
    x_plot = np.linspace(min(tmdf['MRKR_NBR']), max(tmdf['MRKR_NBR']), int((max(tmdf['MRKR_NBR'])-min(tmdf['MRKR_NBR']))*10+1)).reshape(-1, 1)
    xs = np.linspace(min(Xlist), max(Xlist), int((max(Xlist)-min(Xlist))*10+1))
    log_dens = kde.score_samples(x_plot)
    density = np.exp(log_dens)

    data = pd.DataFrame({'Mile': xs, 'YValue': density})
    coordinates = []

    for i in range(len(tmdf)-1):
        mile1, mile2 = tmdf.iloc[i]['MRKR_NBR'], tmdf.iloc[i + 1]['MRKR_NBR']
        coord1, coord2 = tmdf.iloc[i]['geometry'], tmdf.iloc[i + 1]['geometry']
        for mile in xs:
            if mile1 <= mile < mile2:
                coordinates.append(interpolate_coordinates(mile1, mile2, coord1, coord2, mile))
    coordinates.append(Point(coord2.x,coord2.y))

    data['Coordinates'] = coordinates
    data['Road'] = road

    tvdf['mile'] = tvdf.apply(lambda row: closest_mile_marker(row['X'], row['Y'], data), axis=1)
    tvdf = tvdf.sort_values('mile')

    tvdf = tvdf[~tvdf['T_FLAG'].str.endswith('SR')]

    tvdf['first_non_num'] = tvdf['T_FLAG'].apply(first_non_number)
    tvdf = tvdf[tvdf['first_non_num'] != 'R']
    tvdf.drop(columns=['first_non_num'], inplace=True)

    vs = tvdf[['AADT_2022','mile']]
    vs = vs.dropna()
    vs = vs.groupby('mile', as_index=False).agg({'AADT_2022':'mean'})

    data = data.merge(vs,left_on='Mile',right_on='mile',how='left')
    data = data.drop('mile',axis=1)
    data['AADT_2022'] = data['AADT_2022'].interpolate()

    data = data.dropna()
    data['z'] = data['YValue']/data['AADT_2022']
    dfs.append(data)


BS0123
bad df
SH0141
SH0146


/var/folders/fz/qwk5rxqs39gcy8h_0lhy47c80000gn/T/ipykernel_40576/4022452298.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tvdf['mile'] = tvdf.apply(lambda row: closest_mile_marker(row['X'], row['Y'], data), axis=1)
/var/folders/fz/qwk5rxqs39gcy8h_0lhy47c80000gn/T/ipykernel_40576/4022452298.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tvdf['mile'] = tvdf.apply(lambda row: closest_mile_marker(row['X'], row['Y'], data), axis=1)
/var/folders/fz/qwk5rxqs39gcy8h_0lhy47c80000gn/T/ipykernel_40576/402

FM1584
FM0514
FM0073
FM1788


/var/folders/fz/qwk5rxqs39gcy8h_0lhy47c80000gn/T/ipykernel_40576/4022452298.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tvdf['mile'] = tvdf.apply(lambda row: closest_mile_marker(row['X'], row['Y'], data), axis=1)
/var/folders/fz/qwk5rxqs39gcy8h_0lhy47c80000gn/T/ipykernel_40576/4022452298.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tvdf['mile'] = tvdf.apply(lambda row: closest_mile_marker(row['X'], row['Y'], data), axis=1)


FM0417
US0070


...

/var/folders/fz/qwk5rxqs39gcy8h_0lhy47c80000gn/T/ipykernel_40576/4022452298.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tvdf['mile'] = tvdf.apply(lambda row: closest_mile_marker(row['X'], row['Y'], data), axis=1)


US0290


/var/folders/fz/qwk5rxqs39gcy8h_0lhy47c80000gn/T/ipykernel_40576/4022452298.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tvdf['mile'] = tvdf.apply(lambda row: closest_mile_marker(row['X'], row['Y'], data), axis=1)
/var/folders/fz/qwk5rxqs39gcy8h_0lhy47c80000gn/T/ipykernel_40576/4022452298.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tvdf['mile'] = tvdf.apply(lambda row: closest_mile_marker(row['X'], row['Y'], data), axis=1)
/var/folders/fz/qwk5rxqs39gcy8h_0lhy47c80000gn/T/ipykernel_40576/402

RM0864
FM0267
FM1058
FM1275


/var/folders/fz/qwk5rxqs39gcy8h_0lhy47c80000gn/T/ipykernel_40576/4022452298.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tvdf['mile'] = tvdf.apply(lambda row: closest_mile_marker(row['X'], row['Y'], data), axis=1)


In [4]:
bigdf = dfs[0]
dfs = dfs[1:]
for df in dfs:
    bigdf = pd.concat([bigdf,df],ignore_index=True)
bigdf.to_csv('bigdf.csv')

In [5]:
a=bigdf

In [6]:
bigdf = a
print(len(bigdf.index))
bigdf = bigdf[bigdf['AADT_2022'] > 5000]
print(len(bigdf.index))
quantile_value = bigdf['z'].quantile(0.97)
bigdf = bigdf[bigdf['z'] > quantile_value]

526480
201370


In [7]:
bigdf['latitude'] = bigdf['Coordinates'].apply(lambda point: point.y)
bigdf['longitude'] = bigdf['Coordinates'].apply(lambda point: point.x)

import folium
from folium.plugins import HeatMap
data = bigdf
m = folium.Map(location=[30.26, -97.73], zoom_start=13)

heat_data = [[row['latitude'], row['longitude'], row['z']] for index, row in data.iterrows()]
HeatMap(heat_data).add_to(m)
m.save('heatmap.html')


In [13]:
import matplotlib.colors as mcolors

def get_color(volume, min_vol, max_vol):
    norm = mcolors.Normalize(min_vol, max_vol)
    cmap = plt.get_cmap("viridis")
    rgba = cmap(norm(volume))
    return mcolors.to_hex(rgba)

m = folium.Map(location=[data['latitude'].mean(), data['longitude'].mean()], zoom_start=7)

min_vol = a['z'].min()
max_vol = data['z'].max()

for i in range(len(data) - 1):
    if data['Road'].iloc[i] == data['Road'].iloc[i + 1]:  # Check if points are on the same road
        latlon1 = [data['latitude'].iloc[i], data['longitude'].iloc[i]]
        latlon2 = [data['latitude'].iloc[i + 1], data['longitude'].iloc[i + 1]]
        color = get_color(data['z'].iloc[i], min_vol, max_vol)
        folium.PolyLine(locations=[latlon1, latlon2], color=color, weight=5).add_to(m)

m.save('speed_map.html')


In [17]:
import io
from PIL import Image
import selenium

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('image.png')

In [18]:
import os
from selenium import webdriver
import time
mapFname = 'speed_map.html'
mapUrl = 'file://{0}/{1}'.format(os.getcwd(), mapFname)

# download gecko driver for firefox from here - https://github.com/mozilla/geckodriver/releases

# use selenium to save the html as png image
driver = webdriver.Chrome()
driver.get(mapUrl)
# wait for 5 seconds for the maps and other assets to be loaded in the browser
time.sleep(5)
driver.save_screenshot('output.png')
driver.quit()